In [1]:
import csv
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 

### もしstatsが試合前に分かっていたら

In [2]:
df_match_allyears = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
df_match_allyears["Date"] = pd.to_datetime(df_match_allyears["Date"])

In [3]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances
ID,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199
...,...,...,...,...,...,...,...,...,...,...,...,...
21380720,2021,38,2021-12-04,shimizu-s-pulse,cerezo-osaka,7,20,2,1,1,アイスタ,9668
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,駅スタ,9975
21381131,2021,38,2021-12-04,kashiwa-reysol,oita-trinita,11,31,2,3,2,三協Ｆ柏,6028


In [4]:
df_stats = pd.read_csv("./stats.csv", index_col=0)
df_stats

,HomeShots,HomeShotsOnTarget,HomeShotsFromPK,HomePasses,HomeCrosses,HomeDirectFK,HomeIndirectFK,HomeCK,HomeThrowin,HomeDribbling,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
12011413,8.0,3.0,0.0,284.0,6.0,14.0,2.0,4.0,23.0,14.0,...,17.0,1.0,2.0,0.0,0.0,59.0,10.0,135.0,11.1,58.0
12015401,13.0,3.0,0.0,424.0,16.0,9.0,7.0,2.0,13.0,13.0,...,18.0,2.0,7.0,3.0,0.0,43.0,15.0,146.0,6.2,52.1
12010807,7.0,2.0,1.0,311.0,5.0,17.0,3.0,1.0,21.0,8.0,...,9.0,5.0,1.0,3.0,0.0,55.0,8.0,129.0,7.0,56.4
12010918,18.0,9.0,0.0,502.0,24.0,24.0,2.0,7.0,30.0,8.0,...,35.0,7.0,2.0,4.0,0.0,32.0,13.0,145.0,9.7,47.2
12013320,16.0,6.0,0.0,264.0,12.0,14.0,5.0,6.0,15.0,14.0,...,35.0,2.0,4.0,2.0,0.0,48.0,11.0,139.0,11.5,60.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21380720,20.0,5.0,0.0,337.0,14.0,11.0,0.0,5.0,16.0,14.0,...,23.0,0.0,0.0,2.0,0.0,33.0,6.0,99.0,6.1,61.1
21383318,15.0,2.0,0.0,451.0,16.0,10.0,3.0,3.0,15.0,12.0,...,13.0,1.0,3.0,0.0,0.0,42.0,8.0,122.0,11.5,56.7
21381131,15.0,8.0,0.0,446.0,12.0,10.0,3.0,5.0,21.0,10.0,...,21.0,4.0,3.0,0.0,0.0,39.0,11.0,116.0,9.5,54.8
21382223,9.0,4.0,0.0,516.0,10.0,17.0,1.0,2.0,22.0,5.0,...,23.0,2.0,1.0,0.0,0.0,40.0,6.0,121.0,11.6,47.5


In [5]:
df_rate = pd.read_csv("./rating/all_years.csv", index_col=0, parse_dates=True)
df_rate

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,yokohama-fc,tokushima-vortis,montedio-yamagata
2006-03-04,1500,1500,1500,1500,1500,1350,1350,1500,1350,1500,...,1500,1500,1350,1500,1500,1350,1500,1350,1350,1350
2006-03-05,1500,1500,1500,1500,1500,1350,1350,1500,1350,1500,...,1500,1500,1350,1500,1500,1350,1500,1350,1350,1350
2006-03-06,1508,1500,1508,1508,1492,1350,1350,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1350,1350
2006-03-07,1508,1500,1508,1508,1492,1350,1350,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1350,1350
2006-03-08,1508,1500,1508,1508,1492,1350,1351,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1351,1350,1350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,1593,1523,1588,1721,1481,1382,1449,1509,1499,1496,...,1420,1459,1424,1423,1410,1408,1362,1379,1390,1372
2021-12-02,1593,1523,1588,1721,1481,1382,1449,1509,1499,1496,...,1420,1459,1424,1423,1410,1408,1362,1379,1390,1372
2021-12-03,1593,1523,1588,1721,1481,1382,1449,1509,1499,1496,...,1420,1459,1424,1423,1410,1408,1362,1379,1390,1372
2021-12-04,1593,1523,1588,1721,1481,1382,1449,1509,1499,1496,...,1420,1459,1424,1423,1410,1408,1362,1379,1390,1372


In [6]:
for index,row in df_match_allyears.iterrows():
    home_rate = df_rate.loc[row["Date"], row["Home"]]
    away_rate = df_rate.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeRate"] = home_rate
    df_match_allyears.at[index,"AwayRate"] = away_rate 

In [7]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate
ID,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21380720,2021,38,2021-12-04,shimizu-s-pulse,cerezo-osaka,7,20,2,1,1,アイスタ,9668,1423.0,1508.0
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,駅スタ,9975,1499.0,1545.0
21381131,2021,38,2021-12-04,kashiwa-reysol,oita-trinita,11,31,2,3,2,三協Ｆ柏,6028,1449.0,1423.0


In [8]:
df_ranks= pd.read_csv("./ranks/all_years.csv",  index_col=0, parse_dates=True)
df_ranks

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,yokohama-fc,tokushima-vortis,montedio-yamagata
2006-03-04,1,1,1,1,1,19,19,1,19,1,...,1,1,19,1,1,19,1,19,19,19
2006-03-05,3,1,3,3,3,19,19,3,19,1,...,3,3,19,3,3,19,3,19,19,19
2006-03-06,6,8,2,1,12,19,19,3,19,8,...,3,8,19,14,14,19,17,19,19,19
2006-03-07,6,8,2,1,12,19,19,3,19,8,...,3,8,19,14,14,19,17,19,19,19
2006-03-08,6,8,2,1,12,19,19,3,19,8,...,3,8,19,14,14,19,17,19,19,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,4,6,2,1,12,19,14,9,7,13,...,19,8,19,18,19,19,19,20,17,19
2021-12-02,4,6,2,1,12,19,14,9,7,13,...,19,8,19,18,19,19,19,20,17,19
2021-12-03,4,6,2,1,12,19,14,9,7,13,...,19,8,19,18,19,19,19,20,17,19
2021-12-04,4,6,2,1,12,19,14,9,7,13,...,19,8,19,18,19,19,19,20,17,19


In [9]:
for index,row in df_match_allyears.iterrows():
    home_rank = df_ranks.loc[row["Date"], row["Home"]]
    away_rank = df_ranks.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeRank"] = home_rank
    df_match_allyears.at[index,"AwayRank"] = away_rank

In [10]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRank,AwayRank
ID,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0,1.0,1.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0,3.0,3.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0,3.0,3.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0,3.0,3.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21380720,2021,38,2021-12-04,shimizu-s-pulse,cerezo-osaka,7,20,2,1,1,アイスタ,9668,1423.0,1508.0,15.0,10.0
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,駅スタ,9975,1499.0,1545.0,7.0,3.0
21381131,2021,38,2021-12-04,kashiwa-reysol,oita-trinita,11,31,2,3,2,三協Ｆ柏,6028,1449.0,1423.0,14.0,18.0


In [11]:
df_points = pd.read_csv("./points/all_years.csv" ,index_col=0, parse_dates=True)
df_points

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,yokohama-fc,tokushima-vortis,montedio-yamagata
date,,,,,,,,,,,,,,,,,,,,,
2006-03-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-03-05,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2006-03-06,3,1,3,3,0,0,0,3,0,1,...,3,1,0,0,0,0,0,0,0,0
2006-03-07,3,1,3,3,0,0,0,3,0,1,...,3,1,0,0,0,0,0,0,0,0
2006-03-08,3,1,3,3,0,0,0,3,0,1,...,3,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,66,62,78,91,46,28,41,52,59,43,...,0,53,0,32,0,0,0,27,36,0
2021-12-02,66,62,78,91,46,28,41,52,59,43,...,0,53,0,32,0,0,0,27,36,0
2021-12-03,66,62,78,91,46,28,41,52,59,43,...,0,53,0,32,0,0,0,27,36,0


In [12]:
for index,row in df_match_allyears.iterrows():
    home_points = df_points.loc[row["Date"], row["Home"]]
    away_points = df_points.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomePoints"] = home_points
    df_match_allyears.at[index,"AwayPoints"] = away_points

In [13]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRank,AwayRank,HomePoints,AwayPoints
ID,,,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0,1.0,1.0,0.0,0.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0,3.0,3.0,0.0,0.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0,3.0,3.0,0.0,0.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0,3.0,3.0,0.0,0.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0,3.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21380720,2021,38,2021-12-04,shimizu-s-pulse,cerezo-osaka,7,20,2,1,1,アイスタ,9668,1423.0,1508.0,15.0,10.0,39.0,48.0
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,駅スタ,9975,1499.0,1545.0,7.0,3.0,59.0,70.0
21381131,2021,38,2021-12-04,kashiwa-reysol,oita-trinita,11,31,2,3,2,三協Ｆ柏,6028,1449.0,1423.0,14.0,18.0,41.0,32.0


In [14]:
df_gd = pd.read_csv("./goal_differences/all_years.csv",index_col=0, parse_dates=True)
df_gd

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,yokohama-fc,tokushima-vortis,montedio-yamagata
date,,,,,,,,,,,,,,,,,,,,,
2006-03-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-03-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-03-06,1,0,3,6,-1,0,0,2,0,0,...,2,0,0,-2,-2,0,-3,0,0,0
2006-03-07,1,0,3,6,-1,0,0,2,0,0,...,2,0,0,-2,-2,0,-3,0,0,0
2006-03-08,1,0,3,6,-1,0,0,2,0,0,...,2,0,0,-2,-2,0,-3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,25,7,47,53,0,-30,-18,-4,10,-16,...,0,5,0,-25,0,0,0,-44,-19,0
2021-12-02,25,7,47,53,0,-30,-18,-4,10,-16,...,0,5,0,-25,0,0,0,-44,-19,0
2021-12-03,25,7,47,53,0,-30,-18,-4,10,-16,...,0,5,0,-25,0,0,0,-44,-19,0


In [15]:
for index,row in df_match_allyears.iterrows():
    home_gd = df_gd.loc[row["Date"], row["Home"]]
    away_gd = df_gd.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeGD"] = home_gd
    df_match_allyears.at[index,"AwayGD"] = away_gd

In [16]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRank,AwayRank,HomePoints,AwayPoints,HomeGD,AwayGD
ID,,,,,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0,1.0,1.0,0.0,0.0,0.0,0.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21380720,2021,38,2021-12-04,shimizu-s-pulse,cerezo-osaka,7,20,2,1,1,アイスタ,9668,1423.0,1508.0,15.0,10.0,39.0,48.0,-18.0,-3.0
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,駅スタ,9975,1499.0,1545.0,7.0,3.0,59.0,70.0,10.0,24.0
21381131,2021,38,2021-12-04,kashiwa-reysol,oita-trinita,11,31,2,3,2,三協Ｆ柏,6028,1449.0,1423.0,14.0,18.0,41.0,32.0,-18.0,-25.0


In [17]:
df_allyears = pd.concat([df_match_allyears,df_stats],axis=1)
df_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
6010524,2006,1,2006-03-05,yokohama-fa-marinos,kyoto-sanga,5,24,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011001,2006,1,2006-03-05,sanfrecce-hiroshima,kashima-antlers,10,1,3,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21382223,2021,38,2021-12-04,fc-tokyo,avispa-fukuoka,22,23,0,0,0,...,23.0,2.0,1.0,0.0,0.0,40.0,6.0,121.0,11.6,47.5
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,...,13.0,1.0,3.0,0.0,0.0,42.0,8.0,122.0,11.5,56.7
21383414,2021,38,2021-12-04,yokohama-fc,consadole-sapporo,34,14,0,1,2,...,25.0,2.0,0.0,0.0,0.0,31.0,12.0,124.0,11.3,55.8
21383610,2021,38,2021-12-04,tokushima-vortis,sanfrecce-hiroshima,36,10,2,4,2,...,31.0,3.0,1.0,0.0,0.0,23.0,10.0,114.0,6.1,34.8


In [18]:
df = df_allyears[df_allyears["Year"] >= 2012]
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
12010807,2012,1,2012-03-10,nagoya-grampus-eight,shimizu-s-pulse,8,7,1,0,1,...,9.0,5.0,1.0,3.0,0.0,55.0,8.0,129.0,7.0,56.4
12010918,2012,1,2012-03-10,gamba-osaka,vissel-kobe,9,18,2,3,2,...,35.0,7.0,2.0,4.0,0.0,32.0,13.0,145.0,9.7,47.2
12011003,2012,1,2012-03-10,sanfrecce-hiroshima,urawa-red-diamonds,10,3,1,0,1,...,11.0,1.0,3.0,1.0,0.0,42.0,14.0,128.0,5.5,50.9
12011105,2012,1,2012-03-11,kashiwa-reysol,yokohama-fa-marinos,11,5,3,3,0,...,28.0,2.0,1.0,2.0,0.0,44.0,19.0,154.0,9.1,46.3
12011413,2012,1,2012-03-10,consadole-sapporo,jubilo-iwata,14,13,0,0,0,...,17.0,1.0,2.0,0.0,0.0,59.0,10.0,135.0,11.1,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21382223,2021,38,2021-12-04,fc-tokyo,avispa-fukuoka,22,23,0,0,0,...,23.0,2.0,1.0,0.0,0.0,40.0,6.0,121.0,11.6,47.5
21383318,2021,38,2021-12-04,sagan-tosu,vissel-kobe,33,18,0,2,2,...,13.0,1.0,3.0,0.0,0.0,42.0,8.0,122.0,11.5,56.7
21383414,2021,38,2021-12-04,yokohama-fc,consadole-sapporo,34,14,0,1,2,...,25.0,2.0,0.0,0.0,0.0,31.0,12.0,124.0,11.3,55.8
21383610,2021,38,2021-12-04,tokushima-vortis,sanfrecce-hiroshima,36,10,2,4,2,...,31.0,3.0,1.0,0.0,0.0,23.0,10.0,114.0,6.1,34.8


In [33]:
X_features = [ 
    'Sec', 'HomeRate', 'AwayRate','HomeRank', 'AwayRank', 'HomePoints', 'AwayPoints', 'HomeGD', 'AwayGD','HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK', 'HomePasses',
    'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK','HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
    'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards','Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
    'HomeChanceBuildingRate', 'HomePossession', 'AwayShots','AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
    'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin','AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
    'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries','AwayPenaltyAreaEntries', 
    'AwayAttacks', 'AwayChanceBuildingRate','AwayPossession'
]

accuracies = []
loglosses = []

feature_imps = []

for  year in range(2015, 2021):
    
    print(f"=={year}=================================")
    
    X_train = df[(df["Year"] == (year-3)) | (df["Year"] == (year-2)) | (df["Year"] == (year-1))].dropna(how="any")[X_features].values
    y_train = df[(df["Year"] == (year-3)) | (df["Year"] == (year-2)) | (df["Year"] == (year-1))].dropna(how="any")["W/L"].values
    
    X_test = df[df["Year"] == year][X_features].values
    y_test = df[df["Year"] == year]["W/L"].values
    
    clf = RandomForestClassifier(random_state=1234,max_depth=7)
    clf.fit(X_train, y_train)
    
    y_pred_proba = clf.predict_proba(X_test)
    
    accuracy = clf.score(X_test, y_test)
    accuracies.append(accuracy)
    
    feature_imps.append( clf.feature_importances_.tolist())
    
    logloss = log_loss(y_test,y_pred_proba,labels=[0,1,2])
    loglosses.append(logloss)
    
    print(f"Accuracy : {accuracy}")
    print(f"Log loss : {logloss}")
    print(" ")
    
print(" ")
print("======================================")    
print(f"的中率(Accuracy) : {np.mean(accuracies)}" )
print(f"対数損失(Log loss) : {np.mean(loglosses)}" )

==2015=================================
Accuracy : 0.6176470588235294
Log loss : 0.9228505697894119
 
==2016=================================
Accuracy : 0.6503267973856209
Log loss : 0.90355512352435
 
==2017=================================
Accuracy : 0.630718954248366
Log loss : 0.8856853886786413
 
==2018=================================
Accuracy : 0.6274509803921569
Log loss : 0.8804111590535791
 
==2019=================================
Accuracy : 0.6535947712418301
Log loss : 0.899601869422174
 
==2020=================================
Accuracy : 0.6503267973856209
Log loss : 0.8672150722971782
 
 
的中率(Accuracy) : 0.6383442265795206
対数損失(Log loss) : 0.8932198637942225


In [34]:
pd.DataFrame(np.mean(np.array(feature_imps),axis=0), index = X_features).sort_values(0,ascending = False).head(20)

,0
AwayClearances,0.073533
AwayShotsOnTarget,0.063441
HomeShotsOnTarget,0.061172
HomeClearances,0.055924
HomeRate,0.036407
AwayCrosses,0.031762
AwayRate,0.031650
HomeCrosses,0.031257
HomeThrowin,0.027825
HomeChanceBuildingRate,0.024267


もしstatsが試合前に分かっていたら､6割強の的中率を出すことができる｡クリア数､枠内シュート数が重要な指標になっている｡

そのため､statsを回帰して､その回帰したstatsを学習に使ったら､6割近くの的中率向上が目指せるのでは､､､｡